Mencoba fitur translate

In [55]:
import pyodbc
import re

import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime, timedelta

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords

from googletrans import Translator

from FlaskApp.transform import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)

In [3]:
user = 'huda'
password = 'Vancha12'
host = '127.0.0.1'
port = 1433
database = 'HRSystemDB'


def get_connection():         
    return create_engine(
        url=f"mssql+pyodbc://{user}:{password}@{host}:{port}/{database}?driver=SQL Server",
    )

engine = get_connection()
conn = engine.connect()

In [30]:
df_job = pd.DataFrame(engine.execute(
            """
            SELECT Job.JobID, Job.JobTitle, FunctionPosition.FunctionPositionName, EducationLevel.EducationLevelName, City.Name AS CityName, Province.Name AS ProvinceName, Major.MajorName, Job.Description, Job.Requirement
            FROM (((((Job
            RIGHT JOIN FunctionPosition ON Job.FunctionPositionID = FunctionPosition.FunctionPositionID)
            RIGHT JOIN EducationLevel ON Job.EducationLevelID = EducationLevel.EducationLevelID)
            RIGHT JOIN City ON Job.CityID = City.CityID)
            RIGHT JOIN Province ON Job.ProvinceID = Province.ProvinceID)
            RIGHT JOIN Major ON Job.MajorID = Major.MajorID)
            WHERE JobStatus='Publish'
            """
        ))

In [49]:
def change_slangwords(teks):
    if type(teks) == str:
        teks = teks.split(' ')
    for i in range(len(teks)):
        if teks[i] in slangwords:
            teks[i] = slangwords[teks[i]]
    return teks

def translate_teks(teks):
    result = translator.translate(teks, dest='id').text
    if result != teks:
        print(f'Translating {teks[:25]} ...')
    return result

In [50]:
df_job.fillna('', inplace=True)

translator = Translator(service_urls=['translate.googleapis.com'])

focus_cleansing = df_job.Requirement.copy().map(clean_text).map(maintain_alphabet).map(remove_single).map(remove_morespace).apply(translate_teks).apply(lambda x: x.split(' '))

Translating maximum age years old can ...
Translating candidate must possess at ...
Translating has degree in education m ...
Translating maximum age years old min ...


In [51]:
stopwords = [i[0] for i in pd.read_csv('data/stopwords.csv').astype(str).values] + ['usia', 'maksimal', 'tahun', 'pendidikan', 'minimal', 'jurusan', 'pengalaman', 'sebagai', 'laki', 'perempuan', 'memiliki', 'penampilan', 'terbiasa', 'dibidang', 'disukai', 'dibawah', 'bidang', 'ipk', 'menguasai', 'perkantoran', 'bersedia', 'memiliki', 'diutamakan', 'sd', 'up', 'paham', 'menguasai', 'ga', 'pengumpulan', 'gelar', 'setara', 'bidangnya', 'kemampuan', 'pria', 'pengetahuan', 'pekerja', 'berpenampilan', 'years', 'age', 'old', 'with', 'bachelor', 'degree', 'be', 'must', 'candidate', 'gpa', 'in']

In [52]:
slangwords = {
    'max': 'maksimal',
    'maximum': 'maksimal',
    'thn': 'tahun',
    'th': 'tahun',
    'min': 'minimal',
    'miniman': 'minimal',
    'jurnalid': 'jurnalis',
    'mayob': 'myob',
}

In [54]:
focus_cleansing.apply(lambda x: change_slangwords(x)).apply(lambda x: [i for i in x if i not in stopwords]).values[:15]

array([list(['akuntansi', 'komunikasi', 'hukum', 'sekrestaris', 'direksi', 'displin', 'tanggung', 'jujur', 'inisiatif', 'komunikatif', 'presentation', 'skill', 'detail', 'orientied', 'menarik', 'proposional', 'berbahasa', 'mandarin', 'nilai', 'penempatan', 'surabaya', 'menikah']),
       list(['desain', 'komunikasi', 'visual', 'aplikasi', 'dasar', 'desain', 'corel', 'photshop', 'komputer', 'internet', 'tekanan', 'target', 'bertanggung', 'jujur', 'kreatif']),
       list(['good', 'looking', 'kreatif', 'keras', 'disiplin', 'komunikatif', 'murah', 'senyum']),
       list(['kreatif', 'keras', 'disiplin', 'komunikatif', 'murah', 'senyum']),
       list(['ekonomi', 'akuntansi', 'bahasa', 'inggris', 'aktif', 'pasif', 'good', 'computer', 'skill', 'microsoft', 'office', 'outlook', 'pdf', 'accounting', 'system', 'accurate', 'sap', 'jurnalis', 'zahir', 'myob', 'administrasi', 'transaksi', 'bank', 'pph', 'ayat', 'ppn', 'pb', 'mobile', 'tekanan', 'aplikasi', 'accounting', 'microsoft', 'excel', 'ser

In [28]:
35*6

210

In [122]:
df_job.set_index(['JobID'], inplace=True)
df_job.fillna('', inplace=True)

df_job = df_job.applymap(str.lower)
df_job.EducationLevelName = df_job.EducationLevelName.replace('none', '')

translator = Translator(service_urls=['translate.googleapis.com'])

df_job.JobTitle = df_job.JobTitle.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.FunctionPositionName = df_job.FunctionPositionName.apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.MajorName = df_job.MajorName.apply(lambda x: translator.translate(x, dest='id').text.lower())

df_job.Description = df_job.Description.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())
df_job.Requirement = df_job.Requirement.map(clean_text).apply(lambda x: translator.translate(x, dest='id').text.lower())

In [124]:
df_job['JobTitle'] = df_job.JobTitle.str.cat(
    df_job.FunctionPositionName.str.cat(
        df_job.MajorName
    , sep=' ')
, sep=' ').apply(lambda x: ' '.join(set(x.split(' ')))).map(remove_standalonesymbols).map(remove_parenthesesnumber).map(remove_morespace)

df_job.drop(columns=['FunctionPositionName', 'MajorName'], inplace=True)

In [101]:
for i,j in df_job[['Translated', 'JobTitle']].values:
    print(i, '|', j)

direksi sekretaris | direksi sekretaris
staf desain grafis | design desain staff graphic grafis
toko jurusan semua staf awak kapal | jurusan staff crew semua outlet
toko jurusan sma segala awak kapal Pengelola | jurusan sma crew segala outlet manager
pengawas akuntansi | accounting supervisor akuntansi
hukum Pengelola | hukum manager legal
umum qc lapangan sipil operasi teknik | general qc operation lapangan sipil teknik
pajak pengawas akuntansi | pajak accounting supervisor akuntansi
riset jurusan semua staf perkembangan bisnis | research jurusan staff semua business & development
umum jurusan perselingkuhan semua staf | general jurusan staff semua affair
keuangan staf akuntansi  | staff akuntansi & accounting finance
montir listrik jurusan bm-saya semua spv pemeliharaan | bm-me maintenance jurusan semua spv electician 2
Properti jurusan semua pemasaran mendukung | support property jurusan semua marketing
staf informatika dia teknik | informatika staff it teknik
direksi sekretaris sem

In [97]:
df_job.isna().sum()

JobTitle              0
EducationLevelName    0
CityName              0
ProvinceName          0
Description           0
Requirement           0
Translated            0
dtype: int64